In [ ]:
import holoviews as hv
import hvplot.xarray
hv.extension('bokeh')

In [1]:
import xarray as xr
import holoviews as hv
import hvplot.xarray
hv.extension('bokeh')
import rasterio
import numpy as np
import pandas as pd
import re
from pathlib import Path


def extract_date_band(filepath):
    """Extrait la date (YYYYMMDD) et la bande (B2, B3, etc.) du nom du fichier."""
    match_date = re.search(r"(\d{8})", Path(filepath).stem)
    match_band = re.search(r"(B\d{1,2})", Path(filepath).stem)
    
    if match_date and match_band:
        date = pd.to_datetime(match_date.group(1), format="%Y%m%d")
        band = match_band.group(1)
        return date, band
    else:
        raise ValueError(f"Impossible d'extraire date/bande depuis : {filepath}")

# Liste des fichiers TIF
file_paths = [
    r"E:\Travail\Stage_PNRBrenne\Data\Data_Sentinel\output_pretrait\traitement_20190223_B2_.tif",
    r"E:\Travail\Stage_PNRBrenne\Data\Data_Sentinel\output_pretrait\traitement_20190223_B3_.tif",
    r"E:\Travail\Stage_PNRBrenne\Data\Data_Sentinel\output_pretrait\traitement_20190419_B2_.tif",
    r"E:\Travail\Stage_PNRBrenne\Data\Data_Sentinel\output_pretrait\traitement_20190419_B3_.tif"
]

# Dictionnaire pour stocker les données
data_dict = {}

# Variables pour récupérer les coordonnées x et y
x_coords, y_coords = None, None

for path in file_paths:
    date, band = extract_date_band(path)
    
    with rasterio.open(path) as src:
        image_array = src.read(1).astype(np.float32)  # Convertir en float32 directement
        transform = src.transform
        x_size, y_size = src.width, src.height
        
        if x_coords is None or y_coords is None:
            x_coords = np.arange(x_size) * transform.a + transform.c
            y_coords = np.arange(y_size) * transform.e + transform.f
        
    if date not in data_dict:
        data_dict[date] = {}
    
    data_dict[date][band] = image_array

# Créer les dimensions et le Dataset
dates_sorted = sorted(data_dict.keys())

ds_final = xr.Dataset(
    {
        band: (["time", "y", "x"], np.array([data_dict[date][band] for date in dates_sorted], dtype=np.float32))
        for band in data_dict[dates_sorted[0]].keys()
    },
    coords={
        "time": dates_sorted,
        "x": x_coords,
        "y": y_coords
    }
)

ds_final


<xarray.Dataset>
Dimensions:  (time: 2, y: 6450, x: 5878)
Coordinates:
  * time     (time) datetime64[ns] 2019-02-23 2019-04-19
  * x        (x) float64 5.361e+05 5.361e+05 5.361e+05 ... 5.948e+05 5.948e+05
  * y        (y) float64 6.652e+06 6.652e+06 6.652e+06 ... 6.587e+06 6.587e+06
Data variables:
    B2       (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    B3       (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0